In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn

In [18]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, GridSearchCV
from sklearn.preprocessing import LabelEncoder, normalize, MinMaxScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks, ClusterCentroids
from sklearn.neural_network import MLPClassifier

In [3]:
data = pd.read_csv(r'C:\Users\durga\Desktop\uOttawa\Term1\ML_HernaViktor\Assignments\Assignment1\drug_consumption.csv')

In [4]:
#categorical to numerical
columns = ['Alcohol','Amphet', 'Amyl', 'Benzos', 'Caff', 'Cannabis', 'Choc', 'Coke', 'Crack',
           'Ecstasy', 'Heroin', 'Ketamine', 'Legalh', 'LSD', 'Meth', 'Mushrooms','Nicotine', 'Semer', 'VSA']
for column in columns:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
data

,ID,Age,Gender,Education,Country,Ethnicity,Nscore,Escore,Oscore,Ascore,...,Ecstasy,Heroin,Ketamine,Legalh,LSD,Meth,Mushrooms,Nicotine,Semer,VSA
0,1,0.49788,0.48246,-0.05921,0.96082,0.12600,0.31287,-0.57545,-0.58331,-0.91699,...,0,0,0,0,0,0,0,2,0,0
1,2,-0.07854,-0.48246,1.98437,0.96082,-0.31685,-0.67825,1.93886,1.43533,0.76096,...,4,0,2,0,2,3,0,4,0,0
2,3,0.49788,-0.48246,-0.05921,0.96082,-0.31685,-0.46725,0.80523,-0.84732,-1.62090,...,0,0,0,0,0,0,1,0,0,0
3,4,-0.95197,0.48246,1.16365,0.96082,-0.31685,-0.14882,-0.80615,-0.01928,0.59042,...,0,0,2,0,0,0,0,2,0,0
4,5,0.49788,0.48246,1.98437,0.96082,-0.31685,0.73545,-1.63340,-0.45174,-0.30172,...,1,0,0,1,0,0,2,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880,1884,-0.95197,0.48246,-0.61113,-0.57009,-0.31685,-1.19430,1.74091,1.88511,0.76096,...,0,0,0,3,3,0,0,0,0,5
1881,1885,-0.95197,-0.48246,-0.61113,-0.57009,-0.31685,-0.24649,1.74091,0.58331,0.76096,...,2,0,0,3,5,4,4,5,0,0
1882,1886,-0.07854,0.48246,0.45468,-0.57009,-0.31685,1.13281,-1.37639,-1.27553,-1.77200,...,4,0,2,0,2,0,2,6,0,0
1883,1887,-0.95197,0.48246,-0.61113,-0.57009,-0.31685,0.91093,-1.92173,0.29338,-1.62090,...,3,0,0,3,3,0,3,4,0,0


In [5]:
col_names = ['ID','Age', 'Gender', 'Education', 'Country', 'Ethnicity', 'Nscore',
       'Escore', 'Oscore', 'Ascore', 'Cscore', 'Impulsive', 'SS', 'Crack']
feature_col_names = ['Age', 'Gender', 'Education', 'Country', 'Ethnicity', 'Nscore',
       'Escore', 'Oscore', 'Ascore', 'Cscore', 'Impulsive', 'SS']

In [6]:
#Crack resulted in best performance so dataset d contains features and LSD class
D = data[col_names]
D

,ID,Age,Gender,Education,Country,Ethnicity,Nscore,Escore,Oscore,Ascore,Cscore,Impulsive,SS,Crack
0,1,0.49788,0.48246,-0.05921,0.96082,0.12600,0.31287,-0.57545,-0.58331,-0.91699,-0.00665,-0.21712,-1.18084,0
1,2,-0.07854,-0.48246,1.98437,0.96082,-0.31685,-0.67825,1.93886,1.43533,0.76096,-0.14277,-0.71126,-0.21575,0
2,3,0.49788,-0.48246,-0.05921,0.96082,-0.31685,-0.46725,0.80523,-0.84732,-1.62090,-1.01450,-1.37983,0.40148,0
3,4,-0.95197,0.48246,1.16365,0.96082,-0.31685,-0.14882,-0.80615,-0.01928,0.59042,0.58489,-1.37983,-1.18084,0
4,5,0.49788,0.48246,1.98437,0.96082,-0.31685,0.73545,-1.63340,-0.45174,-0.30172,1.30612,-0.21712,-0.21575,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880,1884,-0.95197,0.48246,-0.61113,-0.57009,-0.31685,-1.19430,1.74091,1.88511,0.76096,-1.13788,0.88113,1.92173,0
1881,1885,-0.95197,-0.48246,-0.61113,-0.57009,-0.31685,-0.24649,1.74091,0.58331,0.76096,-1.51840,0.88113,0.76540,0
1882,1886,-0.07854,0.48246,0.45468,-0.57009,-0.31685,1.13281,-1.37639,-1.27553,-1.77200,-1.38502,0.52975,-0.52593,0
1883,1887,-0.95197,0.48246,-0.61113,-0.57009,-0.31685,0.91093,-1.92173,0.29338,-1.62090,-2.57309,1.29221,1.22470,0


In [7]:
#feature selection - remove least variance
D[feature_col_names].var()

Age          0.771517
Gender       0.232891
Education    0.902649
Country      0.490469
Ethnicity    0.027631
Nscore       0.996215
Escore       0.994903
Oscore       0.992471
Ascore       0.994888
Cscore       0.995051
Impulsive    0.910945
SS           0.928720
dtype: float64

In [8]:
#removing features Gender,Ethnicity(Why?least variance) as the features contributes almost equally for all samples
selected_col_names = ['ID','Age', 'Education', 'Country', 'Nscore',
       'Escore', 'Oscore', 'Ascore', 'Cscore', 'Impulsive', 'SS', 'Crack']
selected_feature_col_names = ['Age', 'Education', 'Country', 'Nscore',
       'Escore', 'Oscore', 'Ascore', 'Cscore', 'Impulsive', 'SS']
D = D[selected_col_names]
D

,ID,Age,Education,Country,Nscore,Escore,Oscore,Ascore,Cscore,Impulsive,SS,Crack
0,1,0.49788,-0.05921,0.96082,0.31287,-0.57545,-0.58331,-0.91699,-0.00665,-0.21712,-1.18084,0
1,2,-0.07854,1.98437,0.96082,-0.67825,1.93886,1.43533,0.76096,-0.14277,-0.71126,-0.21575,0
2,3,0.49788,-0.05921,0.96082,-0.46725,0.80523,-0.84732,-1.62090,-1.01450,-1.37983,0.40148,0
3,4,-0.95197,1.16365,0.96082,-0.14882,-0.80615,-0.01928,0.59042,0.58489,-1.37983,-1.18084,0
4,5,0.49788,1.98437,0.96082,0.73545,-1.63340,-0.45174,-0.30172,1.30612,-0.21712,-0.21575,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1880,1884,-0.95197,-0.61113,-0.57009,-1.19430,1.74091,1.88511,0.76096,-1.13788,0.88113,1.92173,0
1881,1885,-0.95197,-0.61113,-0.57009,-0.24649,1.74091,0.58331,0.76096,-1.51840,0.88113,0.76540,0
1882,1886,-0.07854,0.45468,-0.57009,1.13281,-1.37639,-1.27553,-1.77200,-1.38502,0.52975,-0.52593,0
1883,1887,-0.95197,-0.61113,-0.57009,0.91093,-1.92173,0.29338,-1.62090,-2.57309,1.29221,1.22470,0


In [10]:
#decide if user or not
#make a new column for drug which contain the information that a user is using that drug or not - 0 for non-user and 1 for user
columns=['Crack']
cp=['Category']
for i in range(len(columns)):
    D.loc[((D[columns[i]]==0) | (D[columns[i]]==1)),cp[i]] = 0
    D.loc[((D[columns[i]]==2) | (D[columns[i]]==3) | (D[columns[i]]==4) | (D[columns[i]]==5) | (D[columns[i]]==6)),cp[i]] = 1
D

,ID,Age,Education,Country,Nscore,Escore,Oscore,Ascore,Cscore,Impulsive,SS,Crack,Category
0,1,0.49788,-0.05921,0.96082,0.31287,-0.57545,-0.58331,-0.91699,-0.00665,-0.21712,-1.18084,0,0.0
1,2,-0.07854,1.98437,0.96082,-0.67825,1.93886,1.43533,0.76096,-0.14277,-0.71126,-0.21575,0,0.0
2,3,0.49788,-0.05921,0.96082,-0.46725,0.80523,-0.84732,-1.62090,-1.01450,-1.37983,0.40148,0,0.0
3,4,-0.95197,1.16365,0.96082,-0.14882,-0.80615,-0.01928,0.59042,0.58489,-1.37983,-1.18084,0,0.0
4,5,0.49788,1.98437,0.96082,0.73545,-1.63340,-0.45174,-0.30172,1.30612,-0.21712,-0.21575,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880,1884,-0.95197,-0.61113,-0.57009,-1.19430,1.74091,1.88511,0.76096,-1.13788,0.88113,1.92173,0,0.0
1881,1885,-0.95197,-0.61113,-0.57009,-0.24649,1.74091,0.58331,0.76096,-1.51840,0.88113,0.76540,0,0.0
1882,1886,-0.07854,0.45468,-0.57009,1.13281,-1.37639,-1.27553,-1.77200,-1.38502,0.52975,-0.52593,0,0.0
1883,1887,-0.95197,-0.61113,-0.57009,0.91093,-1.92173,0.29338,-1.62090,-2.57309,1.29221,1.22470,0,0.0


In [11]:
#feature scaling
scaler = MinMaxScaler()
scaler.fit(D)
D_transformed = scaler.transform(D)

In [12]:
D_transformed = pd.DataFrame(D_transformed, columns=list(D.columns.values))
D_transformed

,ID,Age,Education,Country,Nscore,Escore,Oscore,Ascore,Cscore,Impulsive,SS,Crack,Category
0,0.00000,0.409137,0.537681,1.000000,0.560562,0.412116,0.435690,0.367654,0.499040,0.428474,0.224398,0.0,0.0
1,0.00053,0.246475,1.000000,1.000000,0.413474,0.796106,0.762567,0.609827,0.479394,0.337920,0.465658,0.0,0.0
2,0.00106,0.409137,0.537681,1.000000,0.444788,0.622976,0.392939,0.266061,0.353580,0.215401,0.619957,0.0,0.0
3,0.00159,0.000000,0.814329,1.000000,0.492045,0.376883,0.527023,0.585213,0.584415,0.215401,0.224398,0.0,0.0
4,0.00212,0.409137,1.000000,1.000000,0.623275,0.250544,0.456995,0.456454,0.688508,0.428474,0.465658,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880,0.99788,0.000000,0.412820,0.000000,0.336890,0.765875,0.835399,0.609827,0.335773,0.629735,1.000000,0.0,0.0
1881,0.99841,0.000000,0.412820,0.000000,0.477550,0.765875,0.624600,0.609827,0.280854,0.629735,0.710933,0.0,0.0
1882,0.99894,0.246475,0.653938,0.000000,0.682246,0.289795,0.323599,0.244253,0.300104,0.565343,0.388117,0.0,0.0
1883,0.99947,0.000000,0.412820,0.000000,0.649318,0.206510,0.577652,0.266061,0.128634,0.705068,0.825752,0.0,0.0


In [13]:
X = D_transformed[selected_feature_col_names].values
y = D_transformed['Category'].values

In [31]:
#SVM
clf = SVC()
scores = cross_val_score(clf, X, y, cv=10)
scores
#finding default gamma
gamma = 1 / (X.shape[1] * X.var())
gamma

1.7160888682025832

In [32]:
#The mean score and the standard deviation are hence given by:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.90 accuracy with a standard deviation of 0.00


In [33]:
#DT
clf = DecisionTreeClassifier(criterion='gini',max_depth=4,random_state=0)
scores = cross_val_score(clf, X, y, cv=10)
scores

array([0.87830688, 0.8994709 , 0.88359788, 0.8994709 , 0.88888889,
       0.89361702, 0.88829787, 0.88829787, 0.88829787, 0.88297872])

In [34]:
#The mean score and the standard deviation are hence given by:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.89 accuracy with a standard deviation of 0.01


In [35]:
#k-neighbors
clf = KNeighborsClassifier()
scores = cross_val_score(clf, X, y, cv=10)
scores

array([0.89417989, 0.88888889, 0.88359788, 0.91005291, 0.85185185,
       0.90957447, 0.86702128, 0.89893617, 0.89361702, 0.89361702])

In [36]:
#The mean score and the standard deviation are hence given by:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.89 accuracy with a standard deviation of 0.02


In [37]:
#Random forest
clf = RandomForestClassifier(max_depth=2, random_state=0)
scores = cross_val_score(clf, X, y, cv=10)
scores

array([0.8994709 , 0.8994709 , 0.8994709 , 0.8994709 , 0.89417989,
       0.89893617, 0.89893617, 0.89893617, 0.89893617, 0.89893617])

In [38]:
#The mean score and the standard deviation are hence given by:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.90 accuracy with a standard deviation of 0.00


In [39]:
#Oversampling

In [40]:
print('Original dataset shape %s' % Counter(y))

Original dataset shape Counter({0.0: 1694, 1.0: 191})


In [41]:
sm = SMOTE(random_state=42)
X_osam, y_osam = sm.fit_resample(X, y)
print('Resampled dataset shape %s' % Counter(y_osam))

Resampled dataset shape Counter({0.0: 1694, 1.0: 1694})


In [42]:
#SVM
clf = SVC(kernel="linear", C=1,random_state=0)
scores = cross_val_score(clf, X_osam, y_osam, cv=10)
scores

array([0.75811209, 0.83185841, 0.76106195, 0.75516224, 0.5840708 ,
       0.61061947, 0.58702065, 0.62536873, 0.77218935, 0.60946746])

In [43]:
#The mean score and the standard deviation are hence given by:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.69 accuracy with a standard deviation of 0.09


In [44]:
#DT
clf = DecisionTreeClassifier(criterion='gini',max_depth=4,random_state=0)
scores = cross_val_score(clf, X_osam, y_osam, cv=10)
scores

array([0.73451327, 0.80825959, 0.77581121, 0.81120944, 0.65486726,
       0.7079646 , 0.67846608, 0.74926254, 0.82544379, 0.70710059])

In [45]:
#The mean score and the standard deviation are hence given by:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.75 accuracy with a standard deviation of 0.06


In [46]:
#k-neighbors
clf = KNeighborsClassifier()
scores = cross_val_score(clf, X_osam, y_osam, cv=10)
scores

array([0.90265487, 0.88495575, 0.84955752, 0.86430678, 0.80235988,
       0.8259587 , 0.7640118 , 0.84955752, 0.90828402, 0.78402367])

In [47]:
#The mean score and the standard deviation are hence given by:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.84 accuracy with a standard deviation of 0.05


In [48]:
#Random forest
clf = RandomForestClassifier(max_depth=2, random_state=0)
scores = cross_val_score(clf, X_osam, y_osam, cv=10)
scores

array([0.76696165, 0.83775811, 0.78466077, 0.7699115 , 0.63716814,
       0.68141593, 0.67551622, 0.71976401, 0.80177515, 0.68934911])

In [49]:
#The mean score and the standard deviation are hence given by:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.74 accuracy with a standard deviation of 0.06


In [50]:
#undersampling

In [51]:
print('Original dataset shape %s' % Counter(y))

Original dataset shape Counter({0.0: 1694, 1.0: 191})


In [52]:
#TomekLinks
tl = TomekLinks()
X_usam, y_usam = tl.fit_resample(X, y)
print('Resampled dataset shape %s' % Counter(y_usam))
#not a good method for our dataset since our majority and minority class seem to be far-off

Resampled dataset shape Counter({0.0: 1634, 1.0: 191})


In [53]:
#ClusterCentroids
cc = ClusterCentroids(random_state=42)
X_usam, y_usam = cc.fit_resample(X, y)
print('Resampled dataset shape %s' % Counter(y_usam))

Resampled dataset shape Counter({0.0: 191, 1.0: 191})


In [54]:
#SVM
clf = SVC(kernel="linear", C=1,random_state=0)
scores = cross_val_score(clf, X_usam, y_usam, cv=10)
scores

array([0.33333333, 0.43589744, 0.65789474, 0.68421053, 0.73684211,
       0.73684211, 0.84210526, 0.63157895, 0.65789474, 0.78947368])

In [55]:
#The mean score and the standard deviation are hence given by:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.65 accuracy with a standard deviation of 0.15


In [56]:
#DT
clf = DecisionTreeClassifier(criterion='gini',max_depth=4,random_state=0)
scores = cross_val_score(clf, X_usam, y_usam, cv=10)
scores

array([0.64102564, 0.64102564, 0.78947368, 0.84210526, 0.73684211,
       0.81578947, 0.86842105, 0.71052632, 0.76315789, 0.89473684])

In [57]:
#The mean score and the standard deviation are hence given by:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.77 accuracy with a standard deviation of 0.08


In [58]:
#k-neighbors
clf = KNeighborsClassifier()
scores = cross_val_score(clf, X_usam, y_usam, cv=10)
scores

array([0.33333333, 0.48717949, 0.57894737, 0.65789474, 0.63157895,
       0.52631579, 0.71052632, 0.52631579, 0.52631579, 0.57894737])

In [59]:
#The mean score and the standard deviation are hence given by:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.56 accuracy with a standard deviation of 0.10


In [60]:
#Random forest
clf = RandomForestClassifier(max_depth=2, random_state=0)
scores = cross_val_score(clf, X_usam, y_usam, cv=10)
scores

array([0.64102564, 0.53846154, 0.73684211, 0.81578947, 0.89473684,
       0.84210526, 0.81578947, 0.76315789, 0.76315789, 0.81578947])

In [61]:
#The mean score and the standard deviation are hence given by:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.76 accuracy with a standard deviation of 0.10


In [62]:
#MLP on D
clf = MLPClassifier(random_state=1, max_iter=700)
scores = cross_val_score(clf, X, y, cv=10)
scores

array([0.89417989, 0.9047619 , 0.8994709 , 0.8994709 , 0.88888889,
       0.89893617, 0.89893617, 0.89893617, 0.89893617, 0.89361702])

In [63]:
#The mean score and the standard deviation are hence given by:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.90 accuracy with a standard deviation of 0.00


In [67]:
mlp = MLPClassifier(max_iter=1000)
parameter_space = {
    'activation': ['logistic','tanh', 'relu'],
    'solver': ['lbfgs','sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}
clf_t = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=10)
clf_t.fit(X, y)
clf_t.best_params_

{'activation': 'logistic',
 'alpha': 0.0001,
 'learning_rate': 'constant',
 'solver': 'sgd'}

In [65]:
#MLP
#{'activation': 'logistic', 'alpha': 0.0001, 'learning_rate': 'constant', 'solver': 'sgd'}
clf = MLPClassifier(max_iter=3000, activation='logistic', alpha=0.0001, learning_rate='constant',solver='sgd')
scores = cross_val_score(clf, X, y, cv=10)
scores

array([0.8994709 , 0.8994709 , 0.8994709 , 0.8994709 , 0.89417989,
       0.89893617, 0.89893617, 0.89893617, 0.89893617, 0.89893617])

In [66]:
#The mean score and the standard deviation are hence given by:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.90 accuracy with a standard deviation of 0.00


In [69]:
#MLP on DB1
clf = MLPClassifier(random_state=1, max_iter=3000)
scores = cross_val_score(clf, X_osam, y_osam, cv=10)
scores

array([0.78761062, 0.89380531, 0.85545723, 0.86135693, 0.77876106,
       0.820059  , 0.80530973, 0.8259587 , 0.86686391, 0.78402367])

In [70]:
#The mean score and the standard deviation are hence given by:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.83 accuracy with a standard deviation of 0.04


In [72]:
mlp = MLPClassifier(max_iter=3000)
parameter_space = {
    'activation': ['logistic','tanh', 'relu'],
    'solver': ['lbfgs','sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}
clf_t = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=10)
clf_t.fit(X_osamsam, y_osam)
clf_t.best_params_

C:\Users\durga\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


{'activation': 'tanh',
 'alpha': 0.05,
 'learning_rate': 'adaptive',
 'solver': 'lbfgs'}

In [74]:
#MLP
#KeyboardInterrupt: 
#best params: 'activation': tanh','solver': 'lbfgs','alpha': 0.05,'learning_rate': 'adaptive'
clf = MLPClassifier(max_iter=3000, activation='tanh', alpha=0.05, learning_rate='adaptive',solver='lbfgs')
scores = cross_val_score(clf, X_osam, y_osam, cv=10)
scores

C:\Users\durga\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\durga\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


KeyboardInterrupt: 

In [75]:
#The mean score and the standard deviation are hence given by:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.91 accuracy with a standard deviation of 0.03


In [82]:
#MLP on DB2
clf = MLPClassifier(random_state=1, max_iter=5000)
scores = cross_val_score(clf, X_usam, y_usam, cv=10)
scores

array([0.53846154, 0.61538462, 0.65789474, 0.71052632, 0.81578947,
       0.84210526, 0.76315789, 0.73684211, 0.71052632, 0.71052632])

In [83]:
#The mean score and the standard deviation are hence given by:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.71 accuracy with a standard deviation of 0.09


In [88]:
mlp = MLPClassifier(max_iter=5000)
parameter_space = {
    'activation': ['logistic','tanh', 'relu'],
    'solver': ['lbfgs','sgd', 'adam'],
    'alpha': [0.0001, 0.05],
}
clf_t = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=10)
clf_t.fit(X_usam, y_usam)
clf_t.best_params_

{'activation': 'relu', 'alpha': 0.05, 'solver': 'adam'}

In [89]:
#MLP
#best params: 'activation': tanh','solver': 'sgd','alpha': 0.05,
clf = MLPClassifier(max_iter=5000, activation='relu', alpha=0.05, learning_rate='adaptive',solver='sgd')
scores = cross_val_score(clf, X_usam, y_usam, cv=10)
scores

array([0.35897436, 0.51282051, 0.73684211, 0.71052632, 0.76315789,
       0.76315789, 0.84210526, 0.68421053, 0.68421053, 0.78947368])

In [90]:
#The mean score and the standard deviation are hence given by:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.68 accuracy with a standard deviation of 0.14


In [91]:
#GradientBoostingClassifier on D
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0)
scores = cross_val_score(clf, X, y, cv=10)
scores

array([0.88359788, 0.9047619 , 0.8994709 , 0.88359788, 0.87830688,
       0.89361702, 0.85638298, 0.87765957, 0.89361702, 0.86702128])

In [92]:
#The mean score and the standard deviation are hence given by:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.88 accuracy with a standard deviation of 0.01


In [100]:
gbc = GradientBoostingClassifier()
parameters = {
    "n_estimators":[5,50,100],
    "max_depth":[1,9]
}
clf_t = GridSearchCV(gbc,parameters,cv=10)
clf_t.fit(X,y)
clf_t.best_params_

{'max_depth': 1, 'n_estimators': 5}

In [103]:
#GradientBoostingClassifier on D
clf = GradientBoostingClassifier(n_estimators=5, learning_rate=1.0,max_depth=1, random_state=0)
scores = cross_val_score(clf, X, y, cv=10)
scores

array([0.88888889, 0.8994709 , 0.8994709 , 0.9047619 , 0.89417989,
       0.88297872, 0.88297872, 0.89361702, 0.89893617, 0.87234043])

In [104]:
#The mean score and the standard deviation are hence given by:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.89 accuracy with a standard deviation of 0.01


In [105]:
#GradientBoostingClassifier on DB1
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0)
scores = cross_val_score(clf, X_osam, y_osam, cv=10)
scores

array([0.62241888, 0.91445428, 0.94100295, 0.92035398, 0.87905605,
       0.91445428, 0.87020649, 0.89970501, 0.95266272, 0.89053254])

In [106]:
#The mean score and the standard deviation are hence given by:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.88 accuracy with a standard deviation of 0.09


In [113]:
gbc = GradientBoostingClassifier()
parameters = {
    "n_estimators":[100],
    "max_depth":[1,9]
}
clf_t = GridSearchCV(gbc,parameters,cv=10)
clf_t.fit(X,y)
clf_t.best_params_

{'max_depth': 1, 'n_estimators': 100}

In [114]:
#GradientBoostingClassifier on DB1
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0)
scores = cross_val_score(clf, X_osam, y_osam, cv=10)
scores

array([0.62241888, 0.91445428, 0.94100295, 0.92035398, 0.87905605,
       0.91445428, 0.87020649, 0.89970501, 0.95266272, 0.89053254])

In [115]:
#The mean score and the standard deviation are hence given by:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.88 accuracy with a standard deviation of 0.09


In [116]:
#GradientBoostingClassifier on DB2
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0)
scores = cross_val_score(clf, X_usam, y_usam, cv=10)
scores

array([0.94871795, 0.76923077, 0.94736842, 0.97368421, 0.97368421,
       1.        , 0.97368421, 0.92105263, 0.86842105, 0.92105263])

In [117]:
#The mean score and the standard deviation are hence given by:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.93 accuracy with a standard deviation of 0.06


In [121]:
gbc = GradientBoostingClassifier()
parameters = {
    "n_estimators":[100],
    "max_depth":[1,7,9]
}
clf_t = GridSearchCV(gbc,parameters,cv=10)
clf_t.fit(X,y)
clf_t.best_params_

{'max_depth': 1, 'n_estimators': 100}

In [122]:
#GradientBoostingClassifier on DB2
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0)
scores = cross_val_score(clf, X_usam, y_usam, cv=10)
scores

array([0.94871795, 0.76923077, 0.94736842, 0.97368421, 0.97368421,
       1.        , 0.97368421, 0.92105263, 0.86842105, 0.92105263])

In [123]:
#The mean score and the standard deviation are hence given by:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.93 accuracy with a standard deviation of 0.06
